In [1]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
from IPython.display import HTML

**Executar este comando quando ocorre um erro ao tentar realizar uma query**

In [9]:
cur.execute('''ROLLBACK;''')

**Criar a ligação com a database**

In [2]:
# def create_connection(db_name, db_user, db_password, db_host, db_port):
#     connection = None
#     try:
#         connection = psycopg2.connect(
#             database=db_name,
#             user=db_user,
#             password=db_password,
#             host=db_host,
#             port=db_port,
#         )
#         print("Connection to PostgreSQL DB successful")
#     except OperationalError as e:
#         print(f"The error '{e}' occurred")
#     return connection
# 
# connection = create_connection("aim4ps", "myuser", "1234", "localhost", "5434")

conn = psycopg2.connect(
    host="localhost",
    port = 5434,
    database="aim4ps",
    user="myuser",
    password="1234")

cur = conn.cursor()

<br>

**Selecionar nomes das colunas**

In [3]:
cur.execute('''
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'CONTRACTS';
''');

print(cur.fetchall())

[('data_celebracao',), ('preco_contratual',), ('prazo_execucao',), ('criterios_ambientais',), ('data_publicacao',), ('entidade_adjudicante',), ('fundamentacao',), ('entidades_concorrentes',), ('url_entidades_concorrente',), ('url_anuncio',), ('url_peças_procedimento',), ('url_documentos',), ('idcontrato',), ('cpv',), ('n_anuncio',), ('tipo_procedimento',), ('objecto_contrato',), ('nif_entidade_adjudicante',)]


<br>

**Número de contratos total**

In [22]:
cur.execute('''
    SELECT COUNT(*) FROM "public"."CONTRACTS";
''')

print("Número de contratos : ", cur.fetchone()[0])

Número de contratos :  338312


<br>

**Imprimir um contrato da tabela CONTRACTS**

In [122]:
cur.execute('''
    TABLE "CONTRACTS";
''')

print(cur.fetchmany(1))

[('8584386', 'None', 'Consulta Prévia', 'Serviços de manutenção de espaços verdes 2022: incluindo o corte de relva, manutenção do sistema de rede de rega, adubagem, pesticidas e inseticidas, podas de árvores e arbustos, o fornecimento de mão de obra necessária, assim como a maquinaria, transportes, materiais, ferramentas e outros equipamentos e produtos necessários ao correto desenvolvimento dos trabalhos', datetime.date(2022, 1, 2), datetime.date(2022, 1, 2), 7163.16, 364, '510837778', 'União das Freguesias de Margaride (Santa Eulália), Várzea, Lagares, Varziela e Moure', 'Artigo 20.º, n.º 1, alínea c) do Código dos Contratos Públicos', 'José Costa Teixeira (-)|||JARDINAGEM - Miguel Ricardo de Melo Moreira (-)|||Enfeites Prometidos, Jardinagem Unipessoal, Lda. (513718117)', 'https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=561696', 'None', 'None', 'https://www.base.gov.pt/Base4/pt/resultados/?type=doc_documentos&id=1491947&ext=.pdf', False, '77311000-3')]


<br>

**Ordenar tipo de procedimentos por ordem decrescente de contratos realizados**

In [175]:
cur = conn.cursor()
cur.execute('''
        SELECT tipo_procedimento, COUNT (tipo_procedimento)
        FROM "CONTRACTS"
        GROUP BY "tipo_procedimento"
        ORDER BY COUNT(tipo_procedimento) DESC;;''')

df = (pd.DataFrame(cur.fetchall()))
HTML(df.to_html(index=False))

0,1
Ajuste Direto Regime Geral,180328
Consulta Prévia,71791
Concurso público,41578
Ao abrigo de acordo-quadro (art.º 259.º),34498
Ao abrigo de acordo-quadro (art.º 258.º),9137
Concurso limitado por prévia qualificação,582
Consulta Prévia Simplificada,347
Procedimento de negociação,17
Concurso público simplificado,13
"Consulta prévia ao abrigo do artigo 7º da Lei n.º 30/2021, de 21.05",13


<br>

**Criar uma tabela apenas com os Ajustes Diretos em Regime Geral**

In [124]:
cur = conn.cursor()
cur.execute('''
        CREATE TABLE ajustes_diretos AS
        SELECT * 
        FROM "CONTRACTS"
        WHERE tipo_procedimento = 'Ajuste Direto Regime Geral';
''')

DuplicateTable: relation "ajustes_diretos" already exists


<br>

**Selecionar apenas um dos contratos da tabela criada acima**

In [4]:
cur.execute('''
    TABLE "ajustes_diretos";
''')

pd.DataFrame(cur.fetchmany(1))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,8584389,None,Ajuste Direto Regime Geral,Serviços no âmbito da Ação Social: Sinalizar i...,2022-01-02,2022-01-02,10800.0,364,506970892,Freguesia de Ronfe,"Artigo 20.º, n.º 1, alínea d) do Código dos Co...",Joana Maria de Sousa Ferreira Lima de Freitas (-),https://www.base.gov.pt/Base4/pt/detalhe/?type...,None,None,https://www.base.gov.pt/Base4/pt/resultados/?t...,False,85322000-2


<br>

**Ordenar, por ordem decrescente, as empresas com mais ajustes diretos realizados**

In [46]:
cur = conn.cursor()
cur.execute('''
        SELECT entidade_adjudicante, COUNT (entidade_adjudicante)
        FROM "CONTRACTS"
        WHERE tipo_procedimento = 'Ajuste Direto Regime Geral'
        GROUP BY "entidade_adjudicante"
        ORDER BY COUNT(entidade_adjudicante) DESC;''')

print("\n Ajustes Diretos")
HTML(pd.DataFrame(cur.fetchmany(20)).to_html(index=False))


 Ajustes Diretos


0,1
"Instituto Português de Oncologia do Porto Francisco Gentil, E. P. E.",22829
Marinha,4828
"Centro Hospitalar Universitário de Lisboa Norte, E. P. E.",3872
"Centro Hospitalar de Tondela - Viseu, E.P.E.",3559
"Centro Hospitalar e Universitário de Coimbra, E. P. E.",3004
MARINHA - Direção de Navios,2636
Estado-Maior-General das Forças Armadas (EMGFA),2612
"Centro Hospitalar Tondela-Viseu, E. P. E.",2477
"Instituto Português de Oncologia de Lisboa Francisco Gentil, E. P. E.",2476
"Hospital Garcia de Orta, E.P.E.",2436


<br>

**Ordenar, por ordem decrescente, as empresas com mais concursos públicos realizados**

In [47]:
cur = conn.cursor()
cur.execute('''
        SELECT entidade_adjudicante, COUNT (entidade_adjudicante)
        FROM "CONTRACTS"
        WHERE tipo_procedimento = 'Concurso público'
        GROUP BY "entidade_adjudicante"
        ORDER BY COUNT(entidade_adjudicante) DESC;''')

print("\n Concursos Públicos")
HTML(pd.DataFrame(cur.fetchmany(20)).to_html(index=False))


 Concursos Públicos


0,1
Santa Casa da Misericórdia de Lisboa,1401
"Centro Hospitalar Universitário de Lisboa Norte, E. P. E.",668
"Instituto Português de Oncologia de Lisboa Francisco Gentil, E. P. E.",558
"Águas do Norte, S. A.",551
"Centro Hospitalar Universitário de Lisboa Central, E.P.E. (CHULC)",439
"Serviço de Saúde da Região Autónoma da Madeira, EPE-RAM",422
Município de Sintra,416
"Centro Hospitalar Barreiro Montijo, E. P. E.",404
"Centro Hospitalar Universitário do Algarve, E.P.E.",399
MARINHA - Direção de Navios,389


<br>

**Ordenar, por ordem decrescente, as empresas com mais consultas prévias realizados**

In [48]:
cur = conn.cursor()
cur.execute('''
        SELECT entidade_adjudicante, COUNT (entidade_adjudicante)
        FROM "CONTRACTS"
        WHERE tipo_procedimento = 'Consulta Prévia'
        GROUP BY "entidade_adjudicante"
        ORDER BY COUNT(entidade_adjudicante) DESC;''')

print("\n Consultas Prévias")
HTML(pd.DataFrame(cur.fetchmany(20)).to_html(index=False))


 Consultas Prévias


0,1
"Serviço de Saúde da Região Autónoma da Madeira, EPE-RAM",1204
Marinha,1014
"Centro Hospitalar Universitário de Lisboa Norte, E. P. E.",976
MARINHA - Direção de Navios,737
"Centro Hospitalar e Universitário de Coimbra, E. P. E.",565
Município de Oeiras,488
Município de Lisboa,404
Município de Sintra,391
"Centro Hospitalar de Lisboa Ocidental, EPE (CHLO)",368
Universidade do Porto,360


<br>

***

<br>

<h2><center> Forças Armadas</center></h2>

**Marinha**

NIF Marinha : 600012662

In [18]:
cur = conn.cursor()

cur.execute('''
        SELECT *
        FROM "CONTRACTS"
        WHERE entidade_adjudicante = 'Marinha';''')

HTML(pd.DataFrame(cur.fetchmany(1)).to_html(index=False))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
7687933,None,Ajuste Direto Regime Geral,3021000517 - Fornecimento de Bens - Carne Diversa,2021-05-29,2021-01-11,250.33,1,600012662,Marinha,"Artigo 24.º, n.º 1, alínea c) do Código dos Contratos Públicos",António José dos Santos Valada (-),https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=4097517,None,None,None,False,15119000-5


In [19]:
cur.execute('''
        SELECT *
        FROM "CONTRACTS"
        WHERE entidade_adjudicante = 'MARINHA - Direção de Navios';''')

HTML(pd.DataFrame(cur.fetchmany(1)).to_html(index=False))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
8602531,None,Ajuste Direto Regime Geral,3021026969/Fornecimento de Sobressalentes - Juntas para o N.R.P. Figueira da Foz,2022-01-04,2022-01-03,2809.6,105,600012662,MARINHA - Direção de Navios,"Artigo 24.º, n.º 1, alínea e), subalínea ii) do Código dos Contratos Públicos","Auto-Sueco portugal - Veiculos Pesados, SA (505051931)|||AUTO-SUECO PORTUGAL - VEÍCULOS PESADOS, S.A. (505051931)",https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=2944932,None,https://www.acingov.pt/acingovprod/2/index.php/zonaPublica/zona_publica_c/donwloadProcedurePiece/NTI3MzIz,None,False,35520000-5


In [45]:
cur = conn.cursor()

cur.execute('''
      SELECT entidade_adjudicante, COUNT (entidade_adjudicante)
      FROM "CONTRACTS"
      WHERE nif_entidade_adjudicante = '600012662'
      GROUP BY "entidade_adjudicante" 
      ORDER BY COUNT(entidade_adjudicante) DESC;''')

HTML(pd.DataFrame(cur.fetchall()).to_html(index=False))

0,1
Marinha,6350
MARINHA - Direção de Navios,3809
Ministério da Defesa Nacional - Marinha,5


In [44]:
cur = conn.cursor()

cur.execute('''
    SELECT entidade_adjudicante, tipo_procedimento, COUNT(*) as count
    FROM "CONTRACTS"
    WHERE nif_entidade_adjudicante = '600012662'
    GROUP BY entidade_adjudicante, tipo_procedimento
    ORDER BY entidade_adjudicante;''')


HTML(pd.DataFrame(cur.fetchall()).to_html(index=False))

0,1,2
Marinha,Ajuste Direto Regime Geral,4828
Marinha,Ao abrigo de acordo-quadro (art.º 258.º),158
Marinha,Ao abrigo de acordo-quadro (art.º 259.º),16
Marinha,Concurso público,334
Marinha,Consulta Prévia,1014
MARINHA - Direção de Navios,Ajuste Direto Regime Geral,2636
MARINHA - Direção de Navios,Ao abrigo de acordo-quadro (art.º 258.º),24
MARINHA - Direção de Navios,Ao abrigo de acordo-quadro (art.º 259.º),20
MARINHA - Direção de Navios,Concurso público,389
MARINHA - Direção de Navios,Consulta Prévia,737


In [61]:
cur = conn.cursor()

cur.execute('''
    SELECT entidade_adjudicante, entidades_concorrentes, COUNT(*) as count
    FROM "CONTRACTS"
    WHERE nif_entidade_adjudicante = '600012662'
    GROUP BY entidade_adjudicante, entidades_concorrentes
    ORDER BY COUNT(entidades_concorrentes) DESC;''')


HTML(pd.DataFrame(cur.fetchmany(20)).to_html(index=False))

0,1,2
Marinha,Gartécnica - Paulo Sérgio Correia Garcia (-),121
Marinha,"Vodafone Portugal - Comunicações Pessoais, S.A. (502544180)",111
Marinha,"Worldparts , comércio de Equipamentos eléctricos Lda (506047385)",73
Marinha,"PEQUIPMAR - PEÇAS E EQUIPAMENTOS MARÍTIMOS, LDA (505192942)",70
Marinha,"Guerreiro &amp; Mendes - Equipamentos Hoteleiros e Artigos Eléctricos, Lda (508566037)",68
Marinha,NUNO MIGUEL DOS SANTOS PIMENTEL (-),61
Marinha,WORLDPARTS - COMÉRCIO (506047385),61
Marinha,Curiosa Tradição Lda (513597832),60
Marinha,"PARTSUL - Comercio e Serviços, Lda (509590616)",58
Marinha,FRIMAT - EQUIPAMENTOS (501397221),56


<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

**Selecionar todos os contratos realizados pela mesma entidade adjudicante**

In [29]:
cur = conn.cursor()

cur.execute('''
        SELECT *
        FROM "CONTRACTS"
        WHERE nif_entidade_adjudicante = '506970892';''')

HTML(pd.DataFrame(cur.fetchmany(2)).to_html(index=False))

# data_celebracao  preco_contratual  prazo_execucao  criterios_ambientais  data_publicacao  entidade_adjudicante  fundamentacao  entidades_concorrentes  url_entidades_concorrente 
# url_anuncio  url_peças_procedimento  url_documentos  idcontrato  cpv  n_anuncio  tipo_procedimento  objecto_contrato  nif_entidade_adjudicante

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
8584389,None,Ajuste Direto Regime Geral,"Serviços no âmbito da Ação Social: Sinalizar idosos em situações de isolamento/fragilidade sem acesso a respostas sociais e articular o encaminhamento para as respetivas respostas sociais disponíveis. Diagnosticar a rede de suporte familiar e social e desenvolver estratégias de intervenção. Avaliar a situação socioeconómica do idoso e informar e apoiar no acesso a subsídios ou prestações da Segurança Social. Articular com as IPSS, nas diferentes respostas sociais existentes, com a USF e com a família e/ou cuidadores informais, no sentido de estabelecer parcerias no cuidado ao idoso. Avaliações psicológicas e avaliações cognitivas",2022-01-02,2022-01-02,10800.0,364,506970892,Freguesia de Ronfe,"Artigo 20.º, n.º 1, alínea d) do Código dos Contratos Públicos",Joana Maria de Sousa Ferreira Lima de Freitas (-),https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=4011442,None,None,https://www.base.gov.pt/Base4/pt/resultados/?type=doc_documentos&id=1491950&ext=.pdf,False,85322000-2
8584391,None,Consulta Prévia,"Serviços no âmbito administrativo, nomeadamente, coordenação e execução de tarefas relacionadas com o Posto dos CTT e serviços administrativos",2022-01-02,2022-01-02,8460.0,364,506970892,Freguesia de Ronfe,"Artigo 20.º, n.º 1, alínea c) do Código dos Contratos Públicos",Olga Alice Mendes Ferreira Mota (-)|||Fernando Eleutério Carvalho Martins Fernandes (-)|||Victor Miguel Ferreira de Castro (-),https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=1688939,None,None,https://www.base.gov.pt/Base4/pt/resultados/?type=doc_documentos&id=1491952&ext=.pdf,False,75110000-0


***

<br>

# Funções 

In [101]:
def preco_contrato(id_anuncio):
    
    '''
    Função que retorna preço contratual a partir do id do anúncio
    '''
    cur.execute('''
        SELECT preco_contratual
        FROM "CONTRACTS"
        WHERE idcontrato = %s; ''', (id_anuncio,))

    
    return np.asarray(cur.fetchall())


(preco_contrato('8584389'))

array([[10800.]])

In [118]:
def preco_contrato1(id_anuncio):
    
    '''
    Função que retorna preço contratual a partir de uma lista de ids de anúncios
    '''
    
    cur.execute('''
        SELECT preco_contratual
        FROM "CONTRACTS"
        WHERE idcontrato IN %s; ''', (tuple(id_anuncio),))
    
    return np.asarray(cur.fetchall())

In [116]:
def all_ids(): 
    '''
    Função que retorna todos os ids dos contratos
    '''
    cur = conn.cursor()
    cur.execute(''' 
                SELECT idcontrato
                FROM "CONTRACTS";
                ''')
    return (cur.fetchall())

ids = all_ids()

In [117]:
preco_contrato1(ids)

array([[  7163.16],
       [ 11261.16],
       [ 10800.  ],
       ...,
       [153612.  ],
       [764139.9 ],
       [  3259.04]])

In [12]:
cur = conn.cursor()
cur.execute('''
        SELECT objecto_contrato, COUNT (objecto_contrato)
        FROM "CONTRACTS"
        GROUP BY "objecto_contrato"
        ORDER BY COUNT(objecto_contrato) DESC;;''')

df = (pd.DataFrame(cur.fetchmany(10)))
HTML(df.to_html(index=False))

0,1
MEDICAMENTOS,2446
VÁRIOS MEDICAMENTOS,444
REAGENTES,374
Vários Medicamentos,271
Fornecimento de eletricidade em regime de mercado livre para Portugal Continental,257
112/2022,238
"P259-2022-028 - Convite ao abrigo do(s) acordo(s)-quadro CP 2021/6 - Medicamentos do Foro Oncológico e Imunomoduladores, CP 2020/61 - Medicamentos Diversos, CP 2019/55 - Medicamentos Diversos, CP 20",223
"P259-2021-125-Convite ao abrigo do(s) acordo(s)-quadro 2021/6, 2020/5, 2020/61, 2019/40, 2021/47 e 2021/41",201
"Convite ao abrigo dos acordos-quadro 2021/6, 2020/5, 2022/40, 2021/47, 2021/41 e 2021/55",198
Medicamentos,189
